In [ ]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('/home/marco/phd/devel/exotic/exotic/')

import os
import math
import glob

import numpy as np
import pandas as pd
import xarray as xr
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from statsmodels.distributions.empirical_distribution import ECDF
from IPython.display import display, HTML

from lab import analysis
from lab.simulation import forcings, observables

REF_RESP_PATH = [
    '/home/marco/phd/data/response/lorenz96/rk4/SF_8.0_1.0_0/',
    '/home/marco/phd/data/response/lorenz96/rk4/SF_8.0_-1.0_0/'            
]

SAVE_FIG = False

## Plot cumulative distribution approximation

In [ ]:
observable = 'energy'
QUANTILES_PATH = f'/home/marco/phd/data/obs/lorenz96/rk4/CF_8.0/quantiles/obs_lorenz96_rk4_CF_8.0_quantiles_{observable}.nc'

This is the cumulative distribution approximation for single node (local) energy.

In [ ]:
quantiles_da = xr.open_dataarray(QUANTILES_PATH)
quantiles_df = pd.DataFrame(data={'order': quantiles_da.quantile_order, 'value': quantiles_da.values})
display(plt.plot(quantiles_df.value, quantiles_df.order))

Here the linear response to the a linear forcing (parameters specified below) is computed via LRT.

In [ ]:
force_linear_coefficient = 0.03
deactivation_time = 100
forcing = f'LF_8.0_{force_linear_coefficient}_0_{deactivation_time}'
print(f'Forcing: {forcing}')

In [ ]:
obs_all = [f"{observable}_below_{format(np.round(quant, 2), '.2f')}q" for quant in np.arange(0, 1.001, 0.01)]
chi = {}
for obs in obs_all:
    response_p1 = xr.open_dataarray(os.path.join(
        REF_RESP_PATH[0],
        observable,
        f'response_lorenz96_rk4_{obs}_SF_8.0_1.0_0.nc'
    ))
    chi[obs] = analysis.compute_susceptibility(response_p1.values.squeeze())
resp_pred = {}
for obs in obs_all:
    resp_pred[obs] = analysis.compute_response(chi[obs], forcings.LinearForcing(
        linear_coefficient=force_linear_coefficient,
        deactivation_time = deactivation_time
    ))

#### Plot estimated CDF and PDF evolution

Below, a 3D plot showing time evolution of the local energy CDF for the selected linear forcing is shown.

In [ ]:
# Quantiles orders
quant_order_rest = np.array(quantiles_df.order)

# Response: variation of frequency of events (local energy values) below each quantile values (at rest)
resp = [resp_pred[obs][0:10000] for obs in obs_all]
resp = np.array(resp)

# Adding response to quantile orders at rest corresponding to selected quantile values, we obtain response
# in terms of frequency (not variation of frequency) of events below those quantile values.
for col in range(0, resp.shape[1]):
    resp[:, col] = resp[:, col] + quant_order_rest

z = resp
y = quantiles_df.value.values
x = np.arange(0, 100, 0.01)
fig_cdf = go.Figure(data=[go.Surface(z=z, x=x, y=y)])
fig_cdf.update_layout(
    title='PDF evolution',
    scene = dict(
        xaxis_title='time',
        yaxis_title='local energy',
        zaxis_title='CDF'
    ),
    autosize=False, width=700, height=700
)

Plot PDF evolution

In [ ]:
# Estimate PDF from CDF
y_pdf = y[1:] - y[:-1]/2
dy = y[1:] - y[:-1]
dy_matrix = np.array([dy, ]*10000).transpose()
z_pdf = (z[1:, :] - z[:-1, :])/dy_matrix
# Time
x = np.arange(0, 100, 0.01)

In [ ]:
fig_pdf = go.Figure(data=[go.Surface(z=z_pdf, x=x, y=y_pdf)])
fig_pdf.update_layout(
    title='PDF evolution',
    scene = dict(
        xaxis_title='time',
        yaxis_title='local energy',
        zaxis_title='PDF'
    ),
    autosize=False, width=700, height=700
)

Import simulated data and compute ECDF

In [ ]:
SIM_PATH = f'/home/marco/phd/data/sim/lorenz96/rk4/{forcing}/'
da_sim = xr.open_mfdataset(glob.glob(os.path.join(SIM_PATH, '*tbr0.01*')), combine='nested', concat_dim='id')
da_sim = da_sim.load()

In [ ]:
energy_obs = observables.Energy()
energy_sim = energy_obs(da_sim)
energy_sim = energy_sim['var']
energy_sim

Let's bin over time axis (we don't need such a time resolution, and we can safely assume time evolution is slow enough)

In [ ]:
def unstack_time_steps(da, bins: int):
    number_of_timesteps = len(energy_sim['time_step'])
    number_of_stacks = int(math.floor(number_of_timesteps/bins))
    time_steps = list(energy_sim.time_step.values)
    new_coords = time_steps[0::number_of_stacks]
    unstacked_dataarrays = []
    for i in range(0, number_of_stacks):
        slice_time_steps = time_steps[i::number_of_stacks]
        da_slice = da.sel(time_step=slice_time_steps)
        da_slice = da_slice.assign_coords({'time_step': new_coords})
        unstacked_dataarrays.append(da_slice)
    return unstacked_dataarrays
unstacked_da = unstack_time_steps(energy_sim, 1000)

In [ ]:
da_concat = xr.concat(unstacked_da, dim='rep')

#### ECDF

In [ ]:
def my_ecdf(da):
    values = da
    print(values)
    return (ECDF(values))
ecdf = da_concat.sel(rep=0, node=0).reduce(func=my_ecdf, dim='time_step')

In [ ]:
ecdf = ECDF(energy_sim.sel(time_step=9999, node=0).values)
plt.plot(ecdf.x, ecdf.y)

In [ ]:
fig_cdf.add_trace(go.Scatter3d(
    x=np.repeat(99.99, len(ecdf.x)), y=ecdf.x, z=ecdf.y,
    marker=dict(
        color='darkblue',
        size=1
    ),
    line=dict(
        color='darkblue',
        width=1
    )
))